# 📓 04_feature_engineering.ipynb (Feature Creation)

In [1]:
# # Feature Engineering
# Create new features from raw data


# ## 1. Import Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("🔧 Starting feature engineering...")

🔧 Starting feature engineering...


In [2]:
# ## 2. Load Processed Data
# Load data with countries
df = pd.read_csv('../data/processed/fraud_data_with_country.csv')
print(f"📊 Data shape: {df.shape}")

# Ensure datetime columns
df['signup_time'] = pd.to_datetime(df['signup_time'])
df['purchase_time'] = pd.to_datetime(df['purchase_time'])

📊 Data shape: (129146, 18)


In [3]:
# ## 3. Time-Based Features
print("⏰ Creating time-based features...")

# 1. Time since signup (in hours)
df['time_since_signup_hours'] = (df['purchase_time'] - df['signup_time']).dt.total_seconds() / 3600

# 2. Extract purchase time components
df['purchase_hour'] = df['purchase_time'].dt.hour
df['purchase_day_of_week'] = df['purchase_time'].dt.dayofweek  # Monday=0, Sunday=6
df['purchase_day_name'] = df['purchase_time'].dt.day_name()
df['purchase_month'] = df['purchase_time'].dt.month
df['purchase_day_of_month'] = df['purchase_time'].dt.day

# 3. Extract signup time components
df['signup_hour'] = df['signup_time'].dt.hour
df['signup_day_of_week'] = df['signup_time'].dt.dayofweek

print(f"✅ Time features created:")
print(f"   - Min time since signup: {df['time_since_signup_hours'].min():.2f} hours")
print(f"   - Max time since signup: {df['time_since_signup_hours'].max():.2f} hours")
print(f"   - Purchase hours: {sorted(df['purchase_hour'].unique())}")


⏰ Creating time-based features...
✅ Time features created:
   - Min time since signup: 0.00 hours
   - Max time since signup: 2879.99 hours
   - Purchase hours: [np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12), np.int32(13), np.int32(14), np.int32(15), np.int32(16), np.int32(17), np.int32(18), np.int32(19), np.int32(20), np.int32(21), np.int32(22), np.int32(23)]


In [4]:
# ## 4. Transaction Velocity Features
print("⚡ Creating transaction velocity features...")

# Sort by user and time for proper calculations
df = df.sort_values(['user_id', 'purchase_time']).reset_index(drop=True)

# Calculate time between consecutive purchases
df['prev_purchase_time'] = df.groupby('user_id')['purchase_time'].shift(1)
df['time_between_purchases_hours'] = (
    df['purchase_time'] - df['prev_purchase_time']
).dt.total_seconds() / 3600

# Count total purchases per user
user_purchase_counts = df.groupby('user_id').size().reset_index(name='total_purchases_per_user')
df = df.merge(user_purchase_counts, on='user_id', how='left')

# Calculate average purchase value per user
user_avg_purchase = df.groupby('user_id')['purchase_value'].mean().reset_index(name='avg_purchase_per_user')
df = df.merge(user_avg_purchase, on='user_id', how='left')

# Create flags based on velocity
df['is_first_purchase'] = df['time_between_purchases_hours'].isna().astype(int)
df['multiple_purchases_flag'] = (df['total_purchases_per_user'] > 1).astype(int)
df['high_frequency_flag'] = (df['time_between_purchases_hours'] < 1).astype(int)  # < 1 hour
df['very_high_frequency_flag'] = (df['time_between_purchases_hours'] < 0.1).astype(int)  # < 6 minutes

print(f"✅ Velocity features created:")
print(f"   - Users with multiple purchases: {df['multiple_purchases_flag'].mean():.2%}")
print(f"   - High frequency transactions: {df['high_frequency_flag'].mean():.2%}")



⚡ Creating transaction velocity features...
✅ Velocity features created:
   - Users with multiple purchases: 0.00%
   - High frequency transactions: 0.00%


In [5]:
# ## 5. Purchase Behavior Features
print("💰 Creating purchase behavior features...")

# 1. Purchase value flags
df['high_value_purchase'] = (df['purchase_value'] > df['purchase_value'].quantile(0.75)).astype(int)
df['very_high_value_purchase'] = (df['purchase_value'] > df['purchase_value'].quantile(0.9)).astype(int)

# 2. Compare with user's average
df['purchase_vs_user_avg'] = df['purchase_value'] / df['avg_purchase_per_user'].replace(0, 1)
df['unusually_large_purchase'] = (df['purchase_vs_user_avg'] > 3).astype(int)

# 3. New customer flags
df['new_customer_1hr'] = (df['time_since_signup_hours'] < 1).astype(int)
df['new_customer_24hr'] = (df['time_since_signup_hours'] < 24).astype(int)
df['new_customer_7days'] = (df['time_since_signup_hours'] < 168).astype(int)

print(f"✅ Behavior features created:")
print(f"   - High value purchases: {df['high_value_purchase'].mean():.2%}")
print(f"   - New customers (<1hr): {df['new_customer_1hr'].mean():.2%}")


💰 Creating purchase behavior features...
✅ Behavior features created:
   - High value purchases: 23.88%
   - New customers (<1hr): 5.18%


In [6]:
# ## 6. Device and Browser Features
print("💻 Creating device and browser features...")

# 1. Device usage patterns
device_counts = df['device_id'].value_counts()
df['device_frequency'] = df['device_id'].map(device_counts)

# Flag suspicious device usage (same device used by many users)
device_user_counts = df.groupby('device_id')['user_id'].nunique().reset_index(name='unique_users_per_device')
df = df.merge(device_user_counts, on='device_id', how='left')
df['suspicious_device'] = (df['unique_users_per_device'] > 3).astype(int)

# 2. Browser risk (from EDA insights)
high_risk_browsers = ['Opera', 'Safari']  # Example from EDA
df['high_risk_browser'] = df['browser'].isin(high_risk_browsers).astype(int)

print(f"✅ Device features created:")
print(f"   - Suspicious devices: {df['suspicious_device'].mean():.2%}")
print(f"   - High risk browsers: {df['high_risk_browser'].mean():.2%}")



💻 Creating device and browser features...
✅ Device features created:
   - Suspicious devices: 5.69%
   - High risk browsers: 18.73%


In [7]:
# ## 7. Location Features
print("📍 Creating location features...")

# Already have: is_high_risk_country
# Create additional location features

# 1. Country transaction frequency
country_counts = df['country'].value_counts()
df['country_frequency'] = df['country'].map(country_counts)

# 2. Flag rare countries
rare_countries = country_counts[country_counts < 10].index
df['rare_country'] = df['country'].isin(rare_countries).astype(int)

print(f"✅ Location features created:")
print(f"   - Rare country transactions: {df['rare_country'].mean():.2%}")


📍 Creating location features...
✅ Location features created:
   - Rare country transactions: 0.16%


In [8]:
# ## 8. Age and Demographic Features
print("👤 Creating demographic features...")

# 1. Age groups
bins = [0, 18, 25, 35, 50, 65, 100]
labels = ['Under 18', '18-24', '25-34', '35-49', '50-64', '65+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

# 2. Suspicious ages
df['suspicious_age'] = ((df['age'] < 13) | (df['age'] > 100)).astype(int)

print(f"✅ Demographic features created:")
print(f"   - Suspicious ages: {df['suspicious_age'].mean():.2%}")


👤 Creating demographic features...
✅ Demographic features created:
   - Suspicious ages: 0.00%


In [9]:
# ## 9. Feature Analysis
print("📊 Analyzing new features...")

# List all new features
new_features = [
    # Time-based
    'time_since_signup_hours', 'purchase_hour', 'purchase_day_of_week',
    'new_customer_1hr', 'new_customer_24hr',
    
    # Velocity
    'time_between_purchases_hours', 'total_purchases_per_user',
    'multiple_purchases_flag', 'high_frequency_flag',
    
    # Purchase behavior
    'high_value_purchase', 'purchase_vs_user_avg',
    'unusually_large_purchase',
    
    # Device/Browser
    'device_frequency', 'suspicious_device', 'high_risk_browser',
    
    # Location
    'is_high_risk_country', 'rare_country',
    
    # Demographic
    'suspicious_age'
]

print(f"🎯 Created {len(new_features)} new features")

# Calculate fraud rate for some key features
print("\n📈 Fraud rates for key features:")
key_features = [
    'new_customer_1hr', 'new_customer_24hr', 
    'high_value_purchase', 'is_high_risk_country',
    'suspicious_device', 'suspicious_age'
]

for feature in key_features:
    if feature in df.columns:
        fraud_rate = df.groupby(feature)['class'].mean()
        print(f"   {feature}:")
        for val, rate in fraud_rate.items():
            print(f"      {val}: {rate:.2%}")


📊 Analyzing new features...
🎯 Created 18 new features

📈 Fraud rates for key features:
   new_customer_1hr:
      0: 4.59%
      1: 99.49%
   new_customer_24hr:
      0: 4.59%
      1: 87.34%
   high_value_purchase:
      0: 9.41%
      1: 9.78%
   is_high_risk_country:
      0: 8.95%
      1: 12.93%
   suspicious_device:
      0: 4.59%
      1: 90.93%
   suspicious_age:
      0: 9.50%


In [10]:
# ## 10. Save Features
print("💾 Saving engineered features...")

# Save full dataset with all features
df.to_csv('../data/processed/fraud_data_with_features.csv', index=False)

# Save feature list
feature_list = {
    'original_features': df.columns.tolist(),
    'new_features': new_features,
    'target_column': 'class'
}

import json
with open('../data/processed/feature_list.json', 'w') as f:
    json.dump(feature_list, f, indent=4)

print("✅ Feature engineering completed!")
print(f"\n📊 Final dataset shape: {df.shape}")
print(f"📋 Total columns: {len(df.columns)}")
print(f"🎯 New features created: {len(new_features)}")
print(f"\n📁 Saved to: ../data/processed/fraud_data_with_features.csv")


💾 Saving engineered features...
✅ Feature engineering completed!

📊 Final dataset shape: (129146, 48)
📋 Total columns: 48
🎯 New features created: 18

📁 Saved to: ../data/processed/fraud_data_with_features.csv
